# このノートブックについて

このノートブックは [Confluent Kafka Python] を利用し、Pythonを用いてApache Kafkaクラスタにデータを書き込むサンプルである。

[Confluent Kafka Python]: https://docs.confluent.io/ja-jp/clients-confluent-kafka-python/1.5.0/overview.html

# 準備

In [1]:
from confluent_kafka import Producer
from confluent_kafka import Consumer
from confluent_kafka import TopicPartition
from confluent_kafka import OFFSET_BEGINNING
from confluent_kafka.admin import AdminClient
from confluent_kafka.admin import NewTopic
import socket
import time

ここでは簡単化のため、単一ノード上にKafka Broker、ZooKeeper、Clientを起動することとする。
そのため、実行ノードの最初にホスト名を取得する。

![Pythonクライアント動作確認のためのイメージ](https://github.com/dobachi/PythonKafkaECHONETLiteExample/blob/main/images/ArchitectureOfKafkaBasic.png?raw=true)

In [2]:
hostname = socket.gethostname()
hostname

'el01'

クライアントに渡すコンフィグを生成する。

In [3]:
producer_conf = {'bootstrap.servers': hostname + ":9092",
                 'client.id': hostname}

In [4]:
consumer_conf = {'bootstrap.servers': hostname + ":9092",
                 'group.id': 0,
                'auto.offset.reset': 'earliest'}

# 実験用のトピックを作る

In [5]:
topic_name = 'test_topic'
num_partition = 1
replication_factor = 1

コンシューマを作成

In [6]:
consumer = Consumer(consumer_conf)

トピックのリストを得るため、メタデータを取得する。

In [7]:
cluster_metadata = consumer.list_topics()
cluster_metadata

ClusterMetadata(UU5KEDIXTJuFCM8nsa7XcQ)

トピックリストを確認

In [8]:
cluster_metadata.topics

{'el_aircon': TopicMetadata(el_aircon, 1 partitions),
 '__consumer_offsets': TopicMetadata(__consumer_offsets, 50 partitions)}

`admin` を取得する。

In [9]:
admin = AdminClient(producer_conf)

トピックを作成する処理を定義

In [10]:
def create_topic(admin, topic_name, num_partition, replication_factor):
    new_topic = NewTopic(topic_name, num_partition, replication_factor)
    result = admin.create_topics([new_topic])
    return result

トピックがあれば削除し、改めて作り直す。

もし既存のトピックを流用したい場合は、冒頭の `#delete_topics` をしている箇所をコメントアウトすること。

In [12]:
if topic_name in cluster_metadata.topics:
    admin.delete_topics([topic_name])

result = create_topic(admin, topic_name, num_partition, replication_factor)   

while(not result[topic_name].done()):
    time.sleep(3)

result

{'test_topic': <Future at 0x7f8ca727bd50 state=finished raised KafkaException>}

In [13]:
consumer.close()

# 書き込む

今回は非同期的に書き込む方式を試す。

In [14]:
producer = Producer(producer_conf)

In [15]:
def acked(err, msg):
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:
        print("Message produced: %s" % (str(msg)))

producer.produce(topic_name, key="key", value="value", callback=acked)

# Wait up to 1 second for events. Callbacks will be invoked during
# this method call if the message is acknowledged.
producer.poll(1)

Message produced: <cimpl.Message object at 0x7f8ca7c320e0>


1

# 読み込む

In [16]:
topic_partition = TopicPartition(topic_name, partition=0, offset=OFFSET_BEGINNING)
topic_partition

TopicPartition{topic=test_topic,partition=0,offset=-2,error=None}

In [17]:
def my_on_assign(consumer, partitions):
    for p in partitions:
         # some starting offset, or use OFFSET_BEGINNING, et, al.
         # the default offset is STORED which means use committed offsets, and if
         # no committed offsets are available use auto.offset.reset config (default latest)
        p.offset = OFFSET_BEGINNING
    # call assign() to start fetching the given partitions.
    consumer.assign(partitions)

In [18]:
try:
    consumer = Consumer(consumer_conf)
    consumer.subscribe([topic_name], on_assign=my_on_assign)
    
    for i in range(10):
        msg = consumer.poll(timeout=1.0)
        if msg is None: continue
        
        if msg.error():
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    # End of partition event
                    sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
                                     (msg.topic(), msg.partition(), msg.offset()))
                elif msg.error():
                    raise KafkaException(msg.error())
        else:
            print('%s : %s' % (msg.key(), msg.value()))
finally:
    consumer.close()

b'key' : b'value'


# 後始末

使用したトピックを消す。もし必要な場合はコメントアウトするなどして残すこと。

In [19]:
admin.delete_topics([topic_name])

{'test_topic': <Future at 0x7f8ca726d610 state=running>}